# Cleaning of the Meta-Learning Wizard-of-Oz pizza ordering dataset

In [ ]:
import re
import ast
from num2words import num2words
import numpy as np

def clean_data(s):
    punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'
    slist = s.split() 
    s = " ".join(x.translate(str.maketrans('', '', punct)).lower().strip() for x in slist if x != "")
    return ' '.join(s.split())

def cleanPM(w):
    t = []
    if ":" in w:
        w = w.split(":")
    for i in w:
        if i.isnumeric():
            t.append(num2words(int(i)).replace("-"," "))
    return t

def pickUp(filename, cleanname):
    yeah = open(filename, "r")
    lines = []
    for line in yeah.readlines():
        crn = []
        words = line.split()
        found = False
        if len(words) < 1:
            crn.append("")
        else:
            for i in range(len(words)-1):
                if found:
                    found = False
                    continue
                if words[i] == "pick" and words[i+1] == "up":
                    crn.append("pickup")
                    found = True
                else:
                    crn.append(words[i])
            if words[-1] != "up":
                crn.append(words[-1])
        lines.append(crn)

    for i in range(len(lines)):
        lines[i] = " ".join(lines[i])
    out_file = open(cleanname, "w")
    out_file.write("\n".join(lines))

myv = np.vectorize(clean_data)

In [14]:
infile = open("ORDER_PIZZA.TXT", "r")
pizza = []
woz = []

for line in infile.readlines():
    pizza.extend(ast.literal_eval(line)['turns'])

for line in pizza:
    temp = []
    for word in line.split():
        word = word.lower()
        if word and word != "(deleted)":
            if re.search(r"\d+[:]{0,1}\d+pm", word):
                temp.extend(cleanPM(word[:-2]))
                temp.append("p m")
            elif word.isnumeric():
                temp.append(num2words(int(word)).replace("-"," "))
            elif re.search(r"[!#$%&()+,-.\/:;<=>?@_`~']{0,2}\d+[!#$%&()+,-.\/:;<=>?@`~']{0,2}\d+[!#$%&()*+,-.\/:;<=>?@`~']{0,2}", word):
                word = re.sub(r'[!"#$%&()\*,-./:;<=>?@^_`{|}~]', " ", word)
                temp.extend(cleanPM(word))
            elif word.lower() == "pm" or word.lower() == "am":
                temp.append("p m")
            else:
                temp.append(word)
    woz.append(" ".join(temp))
    
pizzaclean = myv(woz)

In [15]:
newpp = []
for line in pizzaclean:
    if len(line) == 2 and (line[:2] == "no" or line[:2] == "ok"):
        continue
    elif line[:3] == "yes" or line[:3] == "yep" or line[:3] == "and":
        newpp.append(line[3:])
    elif line[:4] == "yeah" or line[:4] == "okay" or line[:4] == "sure":
        newpp.append(line[4:])
    elif line[:3] == "no " or line[:3] == "ok ":
        newpp.append(line[3:])
    else:
        newpp.append(line)

newpp = [n for n in newpp if n]

In [16]:
with open("woz.txt", "w") as outfile:
    for line in newpp:
        outfile.write(line+"\n")

In [17]:
pickUp('woz.txt','wozpup.txt')